# 01/20

%matplotlib inline

In [1]:
description = 'mobilenet_from_pretrain_v3'

In [2]:

import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from random import shuffle
import math

import pandas as pd

import pickle

from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier,ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing

#from scipy.misc import imread, imresize



In [3]:
import tqdm
import json

In [4]:
train_df = pd.read_csv('../../data/Manually_Annotated_file_lists/training_face_mesh_crop.csv')
train_df.subDirectory_filePath = '../../data/Manually_Annotated_Images_FaceMesh_Cropped/' + train_df.subDirectory_filePath

#train_df_2 = pd.read_csv('../../data/Automatically_annotated_file_list/automatically_annotated_face_mesh_crop.csv')
#train_df_2.subDirectory_filePath = '../../data/Automatically_Annotated_Images_FaceMesh_Cropped/' + train_df_2.subDirectory_filePath
#train_df = train_df.append(train_df_2)
#del train_df_2

train_df = train_df[train_df['have_facemesh']]

In [5]:
from eason_utils import DataFrameBatchIterator
from eason_utils import lprint, now_time_string, log_file_name, change_log_file_name

change_log_file_name(f'''{log_file_name.replace('.log', '')}_{description}.log''')

log_file_name='01_22_23:43:17.log'


In [6]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
config = tf.compat.v1.ConfigProto(gpu_options = tf.compat.v1.GPUOptions(allow_growth = True))
sess = tf.compat.v1.Session(config = config)

In [7]:
mobilenet_pretrained = tf.keras.models.load_model('../models/affectnet_emotions/mobilenet_7.h5')

In [8]:
mobilenet_output = mobilenet_pretrained.get_layer("global_pooling").output

In [9]:
hidden_layers = [256]

In [10]:
valaence_feat = mobilenet_output
for size in hidden_layers:
    valence_feat = tf.keras.layers.Dense(size, activation = 'relu', name = f'feat_valence')(valaence_feat)
outputs_valence = (tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'outputs_valence')(valence_feat) * 4) - 2

arousal_feat = mobilenet_output
for size in hidden_layers:
    arousal_feat = tf.keras.layers.Dense(size, activation = 'relu', name = f'feat_arousal')(arousal_feat)
outputs_arousal = (tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'outputs_arousal')(arousal_feat) * 4) - 2

emotion_feat = mobilenet_output
for size in hidden_layers:
    emotion_feat = tf.keras.layers.Dense(size, activation = 'relu', name = f'feat_emotion')(emotion_feat)
emotions_count = len(np.unique(train_df.expression))
outputs_emotion = tf.keras.layers.Dense(emotions_count, activation = 'softmax', name = 'outputs_emotion')(emotion_feat)

In [11]:
model = tf.keras.Model(inputs=mobilenet_pretrained.input,
                       outputs=(outputs_valence, outputs_arousal, outputs_emotion) , name="mobilenet_train")

In [12]:
model.summary()

Model: "mobilenet_train"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 225, 225, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1 (Conv2D)                 (None, 112, 112, 32  864         ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                    

In [13]:
# Instantiate an optimizer.
optimizer = tf.keras.optimizers.Adam()
# Instantiate a loss function.
MSE_loss = tf.keras.losses.MeanSquaredError()
SCC_loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [14]:
batch_size = 32
epochs = 16

logs = []

In [ ]:
from telegram_notifier import send_message
import time
start_time = time.time()


for epoch in range(epochs):
    total_loss = 0
    total_step = 0
    total_emotion_correct = 0
    total_valence_loss = 0
    total_arousal_loss = 0
    total_emotion_loss = 0
    total_process = 0
    for step, row in enumerate(DataFrameBatchIterator(train_df, batch_size=batch_size)):

        imgs = row.subDirectory_filePath.apply(lambda x: cv2.resize(
            cv2.cvtColor(cv2.imread(x), cv2.COLOR_BGR2RGB), (224, 224)))
        img_array = np.array(list(imgs))

        y_valence = np.array(row.valence)
        y_arousal = np.array(row.arousal)
        y_emotion = np.array(row.expression)

        with tf.GradientTape() as tape:
            logits = model(img_array, training=True)

            pred_valence = logits[0]
            pred_arousal = logits[1]
            pred_emotion = logits[2]

            valence_loss = MSE_loss(y_valence, pred_valence)
            arousal_loss = MSE_loss(y_arousal, pred_arousal)
            emotion_loss = SCC_loss(y_emotion, pred_emotion)

            loss = valence_loss + arousal_loss + emotion_loss

            grads = tape.gradient(loss, model.trainable_weights)

            optimizer.apply_gradients(zip(grads, model.trainable_weights))
        total_loss += float(loss)
        total_step += 1
        
        total_process += len(row)

        valence_loss = float(valence_loss)
        arousal_loss = float(arousal_loss)
        emotion_loss = float(emotion_loss)
        
        total_valence_loss += valence_loss
        total_arousal_loss += arousal_loss
        total_emotion_loss += emotion_loss
        
        emotion_correct = int(sum(pred_emotion.numpy().argmax(axis = 1) == y_emotion))
        total_emotion_correct += emotion_correct
        
        log = {
            'emotion_correct': emotion_correct,
            'valence_loss': valence_loss,
            'arousal_loss': arousal_loss,
            'emotion_loss': emotion_loss,
            'time_take': time.time() - start_time
        }
        log = json.dumps(log)
        lprint(log)
    save_model_path = f"models/{now_time_string}_{description}_epoch{epoch}_batch_{batch_size}"
    model.save(save_model_path)
    lprint(f"Save {save_model_path}")
    send_message(f"Save {save_model_path}\nepoch {epoch} is finish")
    log = {
        'model_path': save_model_path,
        'total_emotion_correct': total_emotion_correct,
        'total_loss': total_loss,
        'total_valence_loss': total_valence_loss,
        'total_arousal_loss': total_arousal_loss,
        'total_emotion_loss': total_emotion_loss,
        'total_process': total_process,
        'time_take': time.time() - start_time
    }
    log = json.dumps(log)
    lprint(log)

{"emotion_correct": 1, "valence_loss": 1.8172329664230347, "arousal_loss": 1.3659603595733643, "emotion_loss": 2.5122873783111572, "time_take": 3.1972382068634033}
{"emotion_correct": 10, "valence_loss": 2.534557819366455, "arousal_loss": 2.4423470497131348, "emotion_loss": 2.122739791870117, "time_take": 3.500185966491699}
{"emotion_correct": 14, "valence_loss": 2.379204511642456, "arousal_loss": 1.736830711364746, "emotion_loss": 1.7736867666244507, "time_take": 3.7280919551849365}
{"emotion_correct": 18, "valence_loss": 1.9437247514724731, "arousal_loss": 1.1171879768371582, "emotion_loss": 1.7552037239074707, "time_take": 4.014060735702515}
{"emotion_correct": 17, "valence_loss": 1.3788342475891113, "arousal_loss": 0.6247645616531372, "emotion_loss": 1.5749136209487915, "time_take": 4.3767781257629395}
{"emotion_correct": 17, "valence_loss": 1.0181050300598145, "arousal_loss": 0.8321330547332764, "emotion_loss": 1.649595856666565, "time_take": 4.616724014282227}
{"emotion_correct":

{"emotion_correct": 13, "valence_loss": 1.095099687576294, "arousal_loss": 1.0274053812026978, "emotion_loss": 1.4648865461349487, "time_take": 16.127872943878174}
{"emotion_correct": 20, "valence_loss": 0.8357353806495667, "arousal_loss": 0.6631572246551514, "emotion_loss": 1.1698901653289795, "time_take": 16.38991117477417}
{"emotion_correct": 15, "valence_loss": 0.8980485796928406, "arousal_loss": 0.706078052520752, "emotion_loss": 1.5663654804229736, "time_take": 16.600515842437744}
{"emotion_correct": 15, "valence_loss": 0.9819141626358032, "arousal_loss": 0.8125412464141846, "emotion_loss": 1.195940613746643, "time_take": 16.8737370967865}
{"emotion_correct": 13, "valence_loss": 0.8631972670555115, "arousal_loss": 0.6595410704612732, "emotion_loss": 1.7667741775512695, "time_take": 17.168280124664307}
{"emotion_correct": 18, "valence_loss": 1.177485466003418, "arousal_loss": 0.876213550567627, "emotion_loss": 1.3248322010040283, "time_take": 17.40528106689453}
{"emotion_correct":

{"emotion_correct": 16, "valence_loss": 1.0501819849014282, "arousal_loss": 0.8314739465713501, "emotion_loss": 1.5765527486801147, "time_take": 27.81778907775879}
{"emotion_correct": 16, "valence_loss": 0.9621105194091797, "arousal_loss": 0.8744242191314697, "emotion_loss": 1.6403563022613525, "time_take": 28.086568355560303}
{"emotion_correct": 21, "valence_loss": 1.0288560390472412, "arousal_loss": 0.8244643211364746, "emotion_loss": 1.092559576034546, "time_take": 28.284438371658325}
{"emotion_correct": 15, "valence_loss": 0.8552061915397644, "arousal_loss": 0.6578271389007568, "emotion_loss": 1.6717792749404907, "time_take": 28.564780950546265}
{"emotion_correct": 13, "valence_loss": 0.8997143507003784, "arousal_loss": 0.8133645057678223, "emotion_loss": 1.6959797143936157, "time_take": 28.785289525985718}
{"emotion_correct": 17, "valence_loss": 0.8706861138343811, "arousal_loss": 0.5929892063140869, "emotion_loss": 1.2487353086471558, "time_take": 29.037788152694702}
{"emotion_co

{"emotion_correct": 18, "valence_loss": 0.9615222215652466, "arousal_loss": 0.8672041893005371, "emotion_loss": 1.6435189247131348, "time_take": 40.301098346710205}
{"emotion_correct": 19, "valence_loss": 1.2191381454467773, "arousal_loss": 1.0335361957550049, "emotion_loss": 1.138778567314148, "time_take": 40.54243493080139}
{"emotion_correct": 22, "valence_loss": 0.827662467956543, "arousal_loss": 0.5733834505081177, "emotion_loss": 1.0488879680633545, "time_take": 40.83232522010803}
{"emotion_correct": 22, "valence_loss": 1.2355034351348877, "arousal_loss": 0.8424631357192993, "emotion_loss": 0.9995455741882324, "time_take": 41.09494638442993}
{"emotion_correct": 18, "valence_loss": 0.9094462394714355, "arousal_loss": 0.7269243597984314, "emotion_loss": 1.2399563789367676, "time_take": 41.347001791000366}
{"emotion_correct": 23, "valence_loss": 0.8508040308952332, "arousal_loss": 0.6736345291137695, "emotion_loss": 1.1414024829864502, "time_take": 41.67621612548828}
{"emotion_correc

{"emotion_correct": 21, "valence_loss": 0.9699397087097168, "arousal_loss": 0.757647693157196, "emotion_loss": 1.155256748199463, "time_take": 53.97740864753723}
{"emotion_correct": 17, "valence_loss": 0.8424865007400513, "arousal_loss": 0.7923278212547302, "emotion_loss": 1.4579368829727173, "time_take": 54.21399402618408}
{"emotion_correct": 18, "valence_loss": 1.0196577310562134, "arousal_loss": 0.93108069896698, "emotion_loss": 1.386181116104126, "time_take": 54.48776626586914}
{"emotion_correct": 17, "valence_loss": 0.8946373462677002, "arousal_loss": 0.7269231081008911, "emotion_loss": 1.208390474319458, "time_take": 54.70667624473572}
{"emotion_correct": 19, "valence_loss": 1.2141170501708984, "arousal_loss": 0.8956661820411682, "emotion_loss": 1.0970829725265503, "time_take": 55.038875102996826}
{"emotion_correct": 24, "valence_loss": 0.936637282371521, "arousal_loss": 0.7238386869430542, "emotion_loss": 0.9982385039329529, "time_take": 55.28449869155884}
{"emotion_correct": 18